### Web Scraper to generate hisotrical weather data

In [1]:
import requests

In [2]:
from bs4 import BeautifulSoup

First, clean up my data in order to identify which dates to scrape weather on

Pulling historic weather data from Pheonix, AZ & Las Vegas, NV

key class items:

date
<div _ngcontent-app-root-c6="" class="date ng-star-inserted">1</div>
<div _ngcontent-app-root-c6="" class="date ng-star-inserted">2</div>
etc, etc.

Weather description for the day
class = <div _ngcontent-app-root-c6="" class="phrase ng-star-inserted">mostly sunny</div>

Hi
<span _ngcontent-app-root-c6="" class="hi">80°</span>

Low
<span _ngcontent-app-root-c6="" class="low">61°</span>

Rain (in inches)
<span _ngcontent-app-root-c12="" class="wu-value wu-value-to">0.21</span>

urls:
LV, NV
https://www.wunderground.com/calendar/us/nv/north-las-vegas/KVGT/date/2015-4 - date will be the changing variable

Phoenix, AZ
https://www.wunderground.com/calendar/us/az/phoenix/KPHX/date/2015-4 - date will be the changing variable